In [1]:
with open('data/ratings_Books.csv', 'r') as f:
    cnt = 0
    for line in f.readlines():
        # user, item, rating, timestamp
        print(line)
        cnt += 1
        if cnt == 10:
            break

AH2L9G3DQHHAJ,0000000116,4.0,1019865600

A2IIIDRK3PRRZY,0000000116,1.0,1395619200

A1TADCM7YWPQ8M,0000000868,4.0,1031702400

AWGH7V0BDOJKB,0000013714,4.0,1383177600

A3UTQPQPM4TQO0,0000013714,5.0,1374883200

A8ZS0I5L5V31B,0000013714,5.0,1393632000

ACNGUPJ3A3TM9,0000013714,4.0,1386028800

A3BED5QFJWK88M,0000013714,4.0,1350345600

A2SUAM1J3GNN3B,0000013714,5.0,1252800000

APOZ15IEYQRRR,0000013714,5.0,1362787200

